In [1]:
import requests as rq
import pandas as pd
from bs4 import BeautifulSoup
# !pip install -U sec-edgar-downloader
from sec_edgar_downloader import Downloader
from tqdm import tqdm
import re
from urllib.parse import urlparse
# import ast
import sys
import json
import os
import gc
import pickle

# Preparation

In [2]:
#API extraction function & Definition of headers:

headers_data = {
    "User-Agent": "Ludwig Kunz (ludwig.kunz@uni-konstanz.de)",
    "Accept-Encoding": "gzip, deflate",
    "Host": "data.sec.gov"
}
headers_doc = {
        "User-Agent": "Ludwig Kunz (ludwig.kunz@uni-konstanz.de)",
        "Accept-Encoding": "gzip, deflate",
        "Host": "www.sec.gov"
}

def get_json(url, headers):
    response = rq.get(url, headers = headers)
    if response.status_code == 200:
        return response.json()
    else:
        raise ImportError(response.status_code)

In [3]:
sp500_changes = pd.read_csv("s&p500_hist_changes.csv") # from https://github.com/fja05680/sp500
sp500_changes["date"] = pd.to_datetime(sp500_changes["date"])
sp500_changes["year"] = sp500_changes["date"].dt.year 
sp500_changes = sp500_changes[((sp500_changes["year"] >= 2005) & (sp500_changes["year"] < 2025))]
sp500_changes.reset_index(drop = True,inplace=True)
sp500_changes["tickers"] = sp500_changes["tickers"].apply(lambda x: x.split(","))

sp500_changes

#iterating over the dataset and creating intersections, to get permanent constituents
common_tickers = set(sp500_changes.iloc[0]["tickers"])
for _, row in sp500_changes[1:].iterrows():
    common_tickers &= set(row["tickers"])
common_tickers = list(common_tickers)

#converting the permanent constituents tickers to ciks
url = "https://www.sec.gov/files/company_tickers.json"
company_tickers = get_json(url, headers_doc)
company_tickers = list(company_tickers.values())
company_tickers = pd.DataFrame(company_tickers)
company_tickers

company_tickers = company_tickers[company_tickers["ticker"].isin(common_tickers)]
company_tickers["cik"] = company_tickers["cik_str"].apply(str).str.zfill(10)
company_tickers = company_tickers[["ticker", "cik"]]
matched_tickers = set(company_tickers["ticker"])
missing_tickers = list(set(common_tickers) - matched_tickers)

df_missing = pd.DataFrame({
    "ticker": missing_tickers,
    "cik": [pd.NA] * len(missing_tickers)
})

company_tickers = pd.concat([company_tickers, df_missing], ignore_index=True)

print("Missing ticker(s):", missing_tickers)

#manually filling the missing CIKs:
company_tickers.loc[company_tickers["ticker"] == "BF.B", "cik"] = "0000014693"
company_tickers.loc[company_tickers["ticker"] == "HES", "cik"] = "0000004447"
company_tickers.loc[company_tickers["ticker"] == "PPL", "cik"] = "0000922224"

# company_tickers = company_tickers[189:] #shortening, for checks and bug fixing
# company_tickers = company_tickers[company_tickers["ticker"] == "ORCL"]

display(company_tickers)

Missing ticker(s): ['HES', 'BF.B']


,ticker,cik
0,NVDA,0001045810
1,MSFT,0000789019
2,AAPL,0000320193
3,JPM,0000019617
4,WMT,0000104169
...,...,...
210,EMN,0000915389
211,APA,0001841666
212,FITB,0000035527
213,HES,0000004447


# Webscraping

In [8]:
# --------------------------Preparation:--------------------------

cols_tokeep = ["accessionNumber", "reportDate", "primaryDocument"]

#link fragmenting and pattern matching functions:

def find_common_ancestor(tag1, tag2):
    parents1 = list(tag1.parents)
    parents2 = list(tag2.parents)

    for p1 in parents1:
        if p1 in parents2:
            return p1
    return None


def extract_section_1A(html, start_href, end_href):
    """
    returns a list of text fragments in section 1A from a BeautifulSoup input of an entire 10-k html
    """
    status = "Starting"
    def fragment(href):
        parsed = urlparse(href)
        return parsed.fragment if parsed.fragment else href.lstrip("#")
    start_id = fragment(start_href)
    end_id = fragment(end_href)

    start_tag = html.find(id = start_id) or html.find("a", attrs = {"name": start_id})
    end_tag = html.find(id = end_id) or html.find("a", attrs = {"name": end_id})

    if not start_tag:
        status = f"Could not locate start anchor #{start_id}"
        content = None
    elif not end_tag: 
        status = f"Could not locate end anchor #{end_id}"
        content = None
    else:
        status = "Worked"

        content = []
        for elem in start_tag.next_elements:
            if elem == end_tag:
                break

            if hasattr(elem, "get_text") and elem.name in ["div", "p", "li", "td", "section"]:
                text = elem.get_text(separator=" ", strip=True)
                if text:
                    content.append(text)
    return content, status

# patterns to find the appropriate sections:
patterns_1A = [re.compile(r'\bITEM\s*1A\b', flags=re.IGNORECASE), re.compile(r'\brisk\s*factors\b', flags=re.IGNORECASE)]
patterns_1B = [re.compile(r'\bITEM\s*1B\b', flags=re.IGNORECASE), re.compile(r'\bunresolved\s*staff\s*comments\b', flags=re.IGNORECASE), re.compile(r'\bunresolved\s*sec\s*staff\s*comments\b', flags=re.IGNORECASE)]

#creating the directory for the extracted sections:
os.makedirs("section_1A", exist_ok = True)

#building the skeleton of the errors dataframe:
errors = pd.DataFrame({
    "Company": pd.Series(dtype="string"),
    "FiscalYearEnd": pd.Series(dtype="datetime64[ns]"),
    "Status": pd.Series(dtype="string"),
    "URL": pd.Series(dtype="string")
})

# --------------------------Full loop:--------------------------
total_documents = 0
for i_comp, row_comp in company_tickers.iterrows():
    status = f'Webscraping for company: {row_comp["ticker"]} ({i_comp+1}/{len(company_tickers)})'
    print('\r\033[K' + status, end='')
    sys.stdout.flush()

    cik = row_comp["cik"]
    cik_no_zeros = str(int(cik))

    # obtaining the document links:
    url = f"https://data.sec.gov/submissions/CIK{cik}.json"
    submissions = get_json(url, headers_data)

    comp_ticker = row_comp["ticker"]
    filings = submissions["filings"]

    recent = pd.DataFrame(filings["recent"])
    recent = recent[recent["form"] == "10-K"].drop(columns = [col for col in recent.columns if col not in cols_tokeep])
    dataframes = [recent]

    for file_dict in filings["files"]:
        filename = file_dict["name"]
        url = f"https://data.sec.gov/submissions/{filename}"
        older_subm = pd.DataFrame(get_json(url, headers_data))
        older_subm = older_subm[older_subm["form"] == "10-K"].drop(columns = [col for col in older_subm.columns if col not in cols_tokeep])
        dataframes.append(older_subm)
    
    df = pd.concat(dataframes, ignore_index = True)
    df["reportDate"] = pd.to_datetime(df["reportDate"])
    df["reportYear"] = df["reportDate"].dt.year
    df = df[df["reportYear"] >= 2005]

    total_documents += len(df)

    df_1A = pd.DataFrame(columns = ["cik", "FiscalYearEnd", "URL", "section_1A"])

    #iterating over all years for the company in question:
    for doc_i, doc_row in tqdm(df.iterrows(), total = len(df)):
        accno = doc_row["accessionNumber"]
        acc_no_no_dash = accno.replace("-", "")
        doclink = doc_row["primaryDocument"]
        url = f'https://www.sec.gov/Archives/edgar/data/{cik_no_zeros}/{acc_no_no_dash}/{doclink}'
        df.at[doc_i, "url"] = url
        
        # downloading the 10-K filing
        response = rq.get(url, headers = headers_doc)
        content = BeautifulSoup(response.content, "html.parser")

        link_1A = None
        link_1B = None
        link_list = [{"link": link, "text": " ".join(link.get_text(separator = " ").split())} for link in content.find_all("a")] 

        # pattern matching for item 1A:
        idx = 0
        while link_1A == None and idx < len(patterns_1A):
            for link_dict in link_list:
                if patterns_1A[idx].search(link_dict["text"]):
                    link_1A = link_dict["link"].get("href")
                    break
            idx += 1
        
        #pattern matching for item 1B:
        idx = 0
        while link_1B == None and idx < len(patterns_1B):
            for link_dict in link_list:
                if patterns_1B[idx].search(link_dict["text"]):
                    link_1B = link_dict["link"].get("href")
                    break
            idx += 1

        #updating the status, using the extraction function, if the link finding worked:
        if link_1A == None:
            sec_1A = None
            extract_status = f'Could not find start link'
        elif link_1B == None:
            sec_1A = None
            extract_status = f'Could not find end link'
        else:
            sec_1A, extract_status = extract_section_1A(content, link_1A, link_1B)
        
        #attempting to find the headings for the sections, in case the link method did not work
        if sec_1A == None:
            bold_tags = content.find_all(["b", "strong"])
            styled_bold = content.find_all(
                lambda tag: tag.has_attr("style") and "font-weight" in tag["style"].lower() and "bold" in tag["style"].lower())
            all_bold = set(bold_tags + styled_bold)

            tag_list = [{"fulltag": tag, "text": " ".join(tag.get_text(separator = " ").split())} for tag in all_bold]
            tag_1A = None
            tag_1B = None

            idx = 0
            while tag_1A == None and idx < len(patterns_1A):
                for tag_dict in tag_list:
                    if patterns_1A[idx].search(tag_dict["text"]):
                        tag_1A = tag_dict["fulltag"]
                        break
                idx += 1
            
            idx = 0
            while tag_1B == None and idx < len(patterns_1B):
                for tag_dict in tag_list:
                    if patterns_1B[idx].search(tag_dict["text"]):
                        tag_1B = tag_dict["fulltag"]
                        break
                idx += 1
            
            if tag_1A == None:
                extract_status = "Bold tag Method: Could not find start tag"
            elif tag_1B == None:
                extract_status = "Bold tag Method: Could not find end tag"
            else:
                sec_1A = []
                for elem in tag_1A.next_elements:
                    if elem == tag_1B:
                        break

                    if hasattr(elem, "get_text") and elem.name in ["div", "p", "li", "td", "section"]:
                        text = elem.get_text(separator=" ", strip=True)
                        if text:
                            sec_1A.append(text)

        elif sec_1A != None and len(sec_1A) == 0:
            extract_status = "Extract function: returned an empty list"    

        #updating the error dict, in case it didn't work
        if sec_1A == None or len(sec_1A) == 0:
            sec_1A = None
            error_row = {"Company": comp_ticker,
                        "FiscalYearEnd": doc_row["reportDate"],
                        "Status": extract_status,
                        "URL": url}
            errors = pd.concat([errors, pd.DataFrame([error_row])], ignore_index = True)
        else:
            #cleaning the list:
            sec_1A = [text for text in sec_1A if(not re.fullmatch(r"\s*\d+\s*", text)) and len(text.strip()) > 5 and text.lower() != "table of contents"]
            #eliminating any exact duplicates that might be left:
            seen = set()
            deduped_text_list = []

            for text in sec_1A:
                if text not in seen:
                    seen.add(text)
                    deduped_text_list.append(text)
            sec_1A = deduped_text_list
            
            df_1A.loc[doc_i, ["cik", "FiscalYearEnd", "URL", "section_1A"]] = [cik, doc_row["reportDate"], url, sec_1A]

        del content
        gc.collect()

    #saving the json file
    with open(f'section_1A/{comp_ticker}.pickle', 'wb') as file:
        pickle.dump(df_1A, file)
    del df_1A

    gc.collect()
    print(f'{len([row for _, row in errors.iterrows() if row["Company"] == comp_ticker])} errors found')

Webscraping for company: NVDA (1/215)

100%|██████████| 21/21 [00:22<00:00,  1.05s/it]

1 errors found
Webscraping for company: MSFT (2/215)


100%|██████████| 21/21 [00:36<00:00,  1.73s/it]

1 errors found
Webscraping for company: AAPL (3/215)

...

0 errors found
Webscraping for company: BF.B (214/215)


100%|██████████| 21/21 [00:15<00:00,  1.36it/s]

7 errors found
Webscraping for company: HES (215/215)


100%|██████████| 20/20 [00:32<00:00,  1.63s/it]

10 errors found


In [9]:
print(f'Errors in total: {len(errors)} out of {total_documents} documents ({((len(errors)/total_documents)*100):.3f}%)')

with open("webscraping_errors.pickle", "wb") as file:
    pickle.dump(errors, file)

Errors in total: 348 out of 4242 documents (8.204%)
